In [ ]:
# Importando bibliotecas

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time

In [ ]:
# Função que altera multiplos caracteres de uma lista

def special_replace(lista, string, special_c = ""):
    for i in lista:
        string = string.replace(i, special_c)
    return string

In [ ]:
# Requisita e retorna o codigo fonte da pagina

def get_data(url):
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    return soup

In [ ]:
# Retorna a quantidade de espisodios do anime e sua data de lançamento em uma lista

def get_info(soup):
    
    info = soup.find('div', class_ = 'information di-ib mt4').text.replace(",", ".")
    indesejados = [' ', 'TV(', 'OVA(', 'Movie(', 'ONA(', 'Special(', ')', 'members', 'eps', '?']
    info = special_replace(indesejados, info)
    info = info.split('\n')[1:-2]
    info[1] = info[1].split('-')[0][3:]
    return info

In [ ]:
# Retorna uma lista com todos os animes no formato de dicionario

def get_animes(soup):
    score = 9
    obras = []
    animes = soup.find_all('tr', class_ = 'ranking-list')
    for anime in animes:
        nome = anime.find('h3', class_ = 'hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3').text
        info = get_info(anime)
        try:
            eps = int(info[0])
        except:
            eps = np.nan
        lancamento = int(info[1])
        while True:
            try:
                nota = float(anime.find('span', class_ = f'text on score-label score-{score}').text)
                break
            except:
                score -= 1
        mais_info = anime.find('h3', class_ = "hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3").a['href']
        
        obra = {
            'Nome': nome,
            'Quantidade de Episódios': eps,
            'Data de lançamento': lancamento,
            'Nota': nota,
            'Mais informações': mais_info
        }
        
        obras.append(obra)
    return obras

In [ ]:
# Salva os animes em um csv

def output(lista_animes):
    df_animes = pd.concat(pd.DataFrame(a) for a in lista_animes)
    df_animes.to_csv('animes_mal.csv', index=False)
    print('Fim. Animes salvos no csv')
    print(df_animes.head())
    return

In [ ]:
# Main

if __name__ == '__main__':
    
    resultados = []
    limit = 600
    tempo = 10
    for i in range(0, limit, 50):    
        url = f"https://myanimelist.net/topanime.php?limit={i}"
        soup = get_data(url)
        resultados.append(get_animes(soup))
        time.sleep(tempo * 60)
    output(resultados)

Fim. Animes salvos no csv
                                 Nome  Quantidade de Episódios  \
0    Fullmetal Alchemist: Brotherhood                     64.0   
1                            Gintama°                     51.0   
2  Shingeki no Kyojin Season 3 Part 2                     10.0   
3                         Steins;Gate                     24.0   
4            Fruits Basket: The Final                     13.0   

   Data de lançamento  Nota                                   Mais informações  
0                2009  9.16  https://myanimelist.net/anime/5114/Fullmetal_A...  
1                2015  9.09       https://myanimelist.net/anime/28977/Gintama°  
2                2019  9.09  https://myanimelist.net/anime/38524/Shingeki_n...  
3                2011  9.09     https://myanimelist.net/anime/9253/Steins_Gate  
4                2021  9.07  https://myanimelist.net/anime/42938/Fruits_Bas...  
